In [1]:
import os
import json
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
# 输入与输出
INPUT_JSONL = '/home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/h5_tienkung_xsens_text_all.jsonl'  # 替换为你的文件路径
OUTPUT_DIR  = '/home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/splits'  # 输出目录（会自动创建）
TRAIN_JSONL = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_text_train.jsonl')
TEST_JSONL  = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_text_test.jsonl')

# 固定测试集 5 个类别（其余全部作为训练）
TEST_CLASSES = {
    'brick_piled_then_press_thrice',
    'plug_pullout_then_press',
    'switch_manipulation',
    'pour_bread_then_place',
    'throw_battery_twice',
}
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    """
    从ID中提取动作类型
    约定: 格式为 数据集名称/动作类型/时间戳 或相似（至少包含两段）
    """
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def split_by_classes(data, test_classes: set):
    """根据类别集合划分 train/test（类别互斥）"""
    train, test = [], []
    for item in data:
        act = extract_action_type(item.get('id', ''))
        if act in test_classes:
            test.append(item)
        else:
            train.append(item)
    return train, test

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def summarize_split(data, title: str):
    """输出该 split 内的基础统计（样本/trajectory/按类别分布）"""
    # 统计
    action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
    for item in data:
        id_string = item.get('id', '')
        action_type = extract_action_type(id_string)
        if action_type:
            # 按你的原始逻辑：以完整 id 作为 trajectory 唯一标识
            action_data[action_type]['trajectories'].add(id_string)
            action_data[action_type]['samples'] += 1

    # 转 DataFrame
    rows = []
    for act, info in action_data.items():
        rows.append({
            '动作类型': act,
            'Trajectory数量': len(info['trajectories']),
            '样本数量': info['samples']
        })
    df = pd.DataFrame(rows).sort_values('样本数量', ascending=False)

    # 汇总
    n_actions = len(df)
    n_traj = int(df['Trajectory数量'].sum()) if not df.empty else 0
    n_samples = int(df['样本数量'].sum()) if not df.empty else 0

    print("\n" + "="*60)
    print(f"📊 {title} 统计")
    print("="*60)
    print(f"  • 动作类型数量: {n_actions}")
    print(f"  • Trajectory总数: {n_traj}")
    print(f"  • 样本总数: {n_samples}")
    if not df.empty:
        print("\n详细统计：")
        print(df.to_string(index=False))
    else:
        print("\n该 split 为空。")
    print("="*60 + "\n")

    return {
        'n_actions': n_actions,
        'n_traj': n_traj,
        'n_samples': n_samples,
        'df': df
    }

def main():
    ensure_dir(OUTPUT_DIR)

    # 1) 读入
    data = load_jsonl(INPUT_JSONL)

    # 2) 切分（类别互斥）
    train, test = split_by_classes(data, TEST_CLASSES)

    # 3) 写出
    dump_jsonl(train, TRAIN_JSONL)
    dump_jsonl(test, TEST_JSONL)
    print(f"✓ 已写出 Train: {len(train)} 条 -> {TRAIN_JSONL}")
    print(f"✓ 已写出 Test : {len(test)} 条 -> {TEST_JSONL}")

    # 4) 统计（分别对 Train/Test）
    train_stat = summarize_split(train, "Train")
    test_stat  = summarize_split(test,  "Test")

    # 5) 汇总比例（便于快速 sanity check）
    total_samples = train_stat['n_samples'] + test_stat['n_samples']
    total_traj    = train_stat['n_traj'] + test_stat['n_traj']
    total_actions = train_stat['n_actions'] + test_stat['n_actions']  # 应为全体类别数量

    print("📌 整体占比（按样本/trajectory/类别数）")
    if total_samples > 0:
        print(f"  • Test 样本占比: {test_stat['n_samples']}/{total_samples} = {test_stat['n_samples']/total_samples:.2%}")
    if total_traj > 0:
        print(f"  • Test Trajectory占比: {test_stat['n_traj']}/{total_traj} = {test_stat['n_traj']/total_traj:.2%}")
    if total_actions > 0:
        print(f"  • Test 类别占比: {test_stat['n_actions']}/{total_actions} = {test_stat['n_actions']/total_actions:.2%}")

    print("\n✅ 切分完成。类别互斥：Test = 5 类；Train = 其余全部类别。")

if __name__ == '__main__':
    main()


✓ 成功加载 5919 条数据
✓ 已写出 Train: 4687 条 -> /home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/splits/h5_tienkung_xsens_text_train.jsonl
✓ 已写出 Test : 1232 条 -> /home/vcj9002/jianshu/workspace/data/robomind/codes/text_demo/splits/h5_tienkung_xsens_text_test.jsonl

📊 Train 统计
  • 动作类型数量: 19
  • Trajectory总数: 830
  • 样本总数: 4687

详细统计：
                                                         动作类型  Trajectory数量  样本数量
                                                   gear_place            52   343
                                                    nut_place            52   339
                                cylinder_pick_box_place_close            58   329
                               battery_insertion_with_pullout            57   328
                                      place_button_then_press            55   321
                                                throw_battery            50   304
                                                seal_stamping            45   290
    